In [1]:
import requests
import pandas as pd
count=0

In [2]:
database_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/"
def mkdir(directory_path):
    directories = directory_path.split("/")
    path=""
    if directories[0]=="":
        directories[0]="root"
    for directory in directories:
        path+=directory+"/"
        r = requests.get(database_endpoint+path+".json")
        if r.json()==None:
            r2 = requests.patch(database_endpoint+path+".json", data='{"%ftype%":"DIRECTORY"}')
mkdir("/user/john/")

In [4]:
# Work on //// at end
# db_endpoint = "https://dsci551-6ec17-default-rtdb.firebaseio.com/root"
# def mkdir(path):
#     if(requests.get(db_endpoint+path+".json").json()==None):
#         requests.patch(db_endpoint+path+".json", data='{"dir":""}')
#     else:
#         print("Directory already exists")
# mkdir("/user/john/a2/b2/c2/a3/z6")

In [28]:
db_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/root"
def ls(path):
    r = requests.get(db_endpoint+path+".json")
    if r.json()==None:
        print("Directory does not exists. Kindly check the path.")
        return
    file_list = []
    print(r.json().keys())
    for directory in r.json().keys():
        if directory == '%ftype%':
                continue
        a = directory.rfind("%dot%")
        if a!=-1:
            directory = directory[0:a]+"."+directory[a+5:]
        print(directory)
        file_list.append(directory)
    return file_list
#         
ls("/user/john")

dict_keys(['%ftype%', 'cutID%dot%csv', 'data%dot%csv'])
cutID.csv
data.csv


['cutID.csv', 'data.csv']

In [4]:
db_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/root"
def rm(path):
    loc = path.rfind(".")
    if loc!=-1:
        path = path[0:loc]+"%dot%"+path[loc+1:]
    if(requests.get(db_endpoint+path+".json").json()==None):
        print("No such file or directory")
        return
    r = requests.delete(db_endpoint+path+".json")
    return path+" deleted successfully"

# rm("/user/john/test")

In [5]:
import math
datanode_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/datanode/"
db_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/root"
def put(file_name, path, k):
    global count
    try:
        df = pd.read_csv(file_name)
    except FileNotFoundError:
        return f"{file_name} not found. Check if the path is correct"
    n=math.ceil(len(df)/k)
    chunks = [df[i:i+n].copy() for i in range(0,df.shape[0],n)]
    for i,chunk in enumerate(chunks):
        count+=1
        name=f"p{i+1}"
        d = chunk.to_json(orient="records")
        data_endpoint=datanode_endpoint+str(count)+".json"
        r = requests.put(data_endpoint, data=d)
        file_name = file_name.replace(".","%dot%")
        partition_str = '{"'+name+'":"'+data_endpoint+'"}'
        r = requests.patch(db_endpoint+path+"/"+file_name+".json", data=partition_str)
    return f"{file_name} has been uploaded to file system in {k} partitions"

In [6]:
put("data.csv", "/user/john", 5)

'data%dot%csv has been uploaded to file system in 5 partitions'

In [7]:
db_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/root"
def getPartitionLocations(path):
    loc = path.rfind(".")
    if loc!=-1:
        path = path[0:loc]+"%dot%"+path[loc+1:]
    r = requests.get(db_endpoint+path+"/.json")
    if r.json() == None:return []
    locations = []
    for partition_num, location in r.json().items():
        locations.append(location)
    return locations

In [54]:
loc = getPartitionLocations("/user/john/data.csv")
print(loc)

['https://dsci551-337de-default-rtdb.firebaseio.com/datanode/7.json', 'https://dsci551-337de-default-rtdb.firebaseio.com/datanode/8.json', 'https://dsci551-337de-default-rtdb.firebaseio.com/datanode/9.json', 'https://dsci551-337de-default-rtdb.firebaseio.com/datanode/10.json', 'https://dsci551-337de-default-rtdb.firebaseio.com/datanode/11.json']


In [8]:
db_endpoint = "https://dsci551-337de-default-rtdb.firebaseio.com/root"
def readPartition(path, partition_num):
    loc = path.rfind(".")
    if loc!=-1:
        path = path[0:loc]+"%dot%"+path[loc+1:]
    partition_str = f"p{partition_num}"
#     print(db_endpoint+path+"/"+partition_str+".json")
    r = requests.get(db_endpoint+path+"/"+partition_str+".json")
    if (r.text)=='null': return 'Incorrect partition number'
    r2 = requests.get(eval(r.text))
    df = pd.DataFrame.from_records(r2.json())
    csv_data = df.to_csv(index=False)
    return csv_data

In [56]:
readPartition("/user/john/data.csv", 3)

'City,Company,Customer ID,Date of Travel,KM Travelled,Transaction ID\nMIAMI FL,Yellow Cab,9449,2016-07-31,20.79,10060456\nMIAMI FL,Yellow Cab,9467,2016-07-31,20.6,10060457\nNASHVILLE TN,Yellow Cab,55969,2016-02-08,15.4,10060458\nNASHVILLE TN,Yellow Cab,56357,2016-07-08,25.25,10060459\nNASHVILLE TN,Yellow Cab,54332,2016-02-08,34.51,10060460\nNASHVILLE TN,Yellow Cab,56830,2016-07-31,33.66,10060461\nNASHVILLE TN,Yellow Cab,55263,2016-06-08,1.9,10060462\nNASHVILLE TN,Yellow Cab,54546,2016-07-08,30.0,10060463\nNASHVILLE TN,Yellow Cab,54469,2016-09-08,36.05,10060464\nNEW YORK NY,Yellow Cab,894,2016-06-08,43.7,10060465\nNEW YORK NY,Yellow Cab,427,2016-04-08,20.8,10060466\nNEW YORK NY,Yellow Cab,1812,2016-07-08,3.06,10060467\nNEW YORK NY,Yellow Cab,2186,2016-07-30,4.28,10060468\nNEW YORK NY,Yellow Cab,177,2016-08-08,33.99,10060469\nNEW YORK NY,Yellow Cab,2251,2016-04-08,14.43,10060470\nNEW YORK NY,Yellow Cab,655,2016-07-30,27.04,10060471\nNEW YORK NY,Yellow Cab,1275,2016-07-30,23.04,10060472\n

In [9]:
import csv
def cat(path):
    locations = getPartitionLocations(path)
    df = pd.DataFrame()
    df_list = []
    for location in locations:
        r = requests.get(location)
        df = pd.DataFrame.from_records(r.json())
        df_list.append(df)
    df_final = pd.concat(df_list)
    csv_data = df_final.to_csv(index=False)
    if df_list == []:return "No such file found. Kindly check the path again"
    df_final = pd.concat(df_list)
    csv_data = df_final.to_csv(index=False)
    return csv_data

In [10]:
# cat("/user/john/data.csv")

## Search Functions

Search method to retrieve the kilo meters travelled by a particular cab in given range of dates.

In [10]:
def mapPartition_getTransacitons(df,cabcompany,date_from,date_to):
    mask = (df['Date of Travel'] > date_from) & (df['Date of Travel'] <= date_to)
    df_mask= df.loc[mask]
    result_df=df_mask[df_mask['Company']==cabcompany]                                            
    return result_df

In [11]:
def reduce_getTransacitons(df_list):
    df_final = pd.concat(df_list)
    df_final=df_final.drop(columns=['Customer ID'])
    return df_final

In [12]:
def search_tarnsactionID(cabcompany,date_from,date_to):
    # Get all partitions of dataset
    locations = getPartitionLocations("/user/john/data.csv")
    df=pd.DataFrame()
    df_list=[]
    # Go through all partitions
    for location in locations:
        # Fetch data for partitions from firebase
        r = requests.get(location)
        # Store data as a dataframe
        df = pd.DataFrame.from_records(r.json())
        # Call mapPartitions() for the dataframe/partition
        output = mapPartition_getTransacitons(df,cabcompany,date_from,date_to)
        # Append the result to df_list
        df_list.append(output)
    # Append all dataframes/partition into single dataframe   
#     print(len(df_final))
    df_final=reduce_getTransacitons(df_list)
    return df_final

In [13]:
result = search_tarnsactionID("Yellow Cab","2016-08-01","2016-08-31")
print(result)

               City     Company Date of Travel  KM Travelled  Transaction ID
4516     ATLANTA GA  Yellow Cab     2016-08-02          7.20        10005915
4532      BOSTON MA  Yellow Cab     2016-08-02         14.30        10005931
4538      BOSTON MA  Yellow Cab     2016-08-02         28.42        10005937
4556      BOSTON MA  Yellow Cab     2016-08-02          7.08        10005955
4560      BOSTON MA  Yellow Cab     2016-08-02         41.04        10005959
...             ...         ...            ...           ...             ...
1752  WASHINGTON DC  Yellow Cab     2016-08-12         31.36        10119921
1757  WASHINGTON DC  Yellow Cab     2016-08-12         18.88        10119926
1758  WASHINGTON DC  Yellow Cab     2016-08-12         41.73        10119927
1762  WASHINGTON DC  Yellow Cab     2016-08-12         44.03        10119931
1768  WASHINGTON DC  Yellow Cab     2016-08-12         30.68        10119937

[6800 rows x 5 columns]


In [14]:
def mapPartition_getCity(df,customerID):
    result_df=df[df['Customer ID']==customerID]['City']                                            
    return result_df  

In [15]:
def reduce_getTransactions(df_list):
    df_final = pd.concat(df_list)
    if df_final.empty: return "No Data Found"
    return df_final

In [16]:
def search_cityofCustomer(customerID):
    # Get all partitions of dataset
#     print(customerID)
    locations = getPartitionLocations("/user/john/data.csv")
    df=pd.DataFrame()
    df_list=[]
    # Go through all partitions
    for location in locations:
        # Fetch data for partitions from firebase
        r = requests.get(location)
        # Store data as a dataframe
        df = pd.DataFrame.from_records(r.json())
        # Call mapPartitions() for the dataframe/partition
        output = mapPartition_getCity(df,customerID)
#         print(output)
        # Append the result to df_list
        df_list.append(output)
    # Append all dataframes/partition into single dataframe   
#     print(len(df_final))
#     print(df_list)
    df_final=reduce_getTransactions(df_list)
    return df_final

In [17]:
search_cityofCustomer(26290)

518      DALLAS TX
11993    DALLAS TX
Name: City, dtype: object

In [18]:
import tkinter as tk
class Shell(tk.Text):
    def __init__(self, parent, **kwargs):
        tk.Text.__init__(self, parent, **kwargs)
        self.bind('<Key>', self.on_key) # setup handler to process pressed keys
        self.cmd = None        # hold the last command issued
        self.show_prompt()

    # to append given text at the end of Text box
    def insert_text(self, txt='', end='\n'):
        self.insert(tk.END, txt+end)
        self.see(tk.END) # make sure it is visible

    def show_prompt(self):
        self.insert_text('$ ', end='')
        self.mark_set(tk.INSERT, tk.END) # make sure the input cursor is at the end
        self.cursor = self.index(tk.INSERT) # save the input position

    # handler to process keyboard input
    def on_key(self, event):
        #print(event)
        if event.keysym == 'Up':
            # show the last command
            self.delete(self.cursor, tk.END)
            self.insert(self.cursor, self.cmd)
            return "break" # disable the default handling of up key
        if event.keysym == 'Down':
            return "break" # disable the default handling of down key
        if event.keysym in ('Left', 'BackSpace'):
            current = self.index(tk.INSERT) # get the current position of the input cursor
            if self.compare(current, '==', self.cursor):
                # if input cursor is at the beginning of input (after the prompt), do nothing
                return "break"
        if event.keysym == 'Return':
            # extract the command input
            cmd = self.get(self.cursor, tk.END).strip()
            self.insert_text() # advance to next line
            self.execute(cmd)
            self.show_prompt()
            if(cmd == 'exit()'):
                self.master.destroy()# quit the shell
            return "break" # disable the default handling of Enter key

    # function to handle edfs commands
    def execute(self, cmd):
        self.cmd = cmd  # save the command
        command = self.cmd.split()
        if command[0] == 'exit()':
            self.insert_text("Exiting shell", end='\n')
        else:
            if command[0] == 'mkdir':
                mkdir(command[1]) #work on dir in mkdir 
            elif command[0] == 'ls':
                result = ls(command[1])
                output = '\n'.join(str(r) for r in result)
                self.insert_text(output, end='\n')
            elif command[0] == 'rm':
                output = rm(command[1])
                self.insert_text(output, end='\n')
            elif command[0] == 'cat':
                output = cat(command[1])
                self.insert_text(output, end='\n')
            else:
                try:
                    output = eval(self.cmd)
                    if isinstance(output, list):
                        output = '\n'.join(str(o) for o in output)
                    self.insert_text(output, end='\n')
                except AttributeError:
                    self.insert_text("Command '%s' not found" % self.cmd, end='\n')
                except SyntaxError:
                    self.insert_text("Kindly check the command and try again", end='\n')

In [19]:
import tkinter as tk
from tkinter import ttk

class MyTreeView(tk.Frame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tvw_results = ttk.Treeview(self, style = 'mystyle.Treeview')
        self.insert = self.tvw_results.insert

        self.tvw_results.grid(row = 0, column = 0, sticky='nsew')
        self.tvw_results.column("#0", minwidth = 1150)
        self.tvw_results.heading('#0', text = 'Found results', anchor = tk.W)

        # Scrollbars + attach scrollbars to TreeView
        sb_vertical = tk.Scrollbar(self, orient = "vertical", command = self.tvw_results.yview)
        sb_horizontal = tk.Scrollbar(self, orient = "horizontal", command = self.tvw_results.xview)
        self.tvw_results.configure(yscrollcommand = sb_vertical.set, xscrollcommand = sb_horizontal.set)
        sb_vertical.grid(row = 0, column = 1, sticky = "ns")
        sb_horizontal.grid(row = 1, column = 0, sticky = "ew")

        # Configure and position grid for TreeView
        self.grid_rowconfigure(0, weight = 1)
        self.grid_columnconfigure(0, weight = 1)

In [25]:
from tkinter import Frame
from tkinter import StringVar, Label, OptionMenu, Entry, Button, Toplevel
import pandastable as pt



class EntryWithPlaceholder(tk.Entry):
    def __init__(self, master=None, placeholder="PLACEHOLDER", color='grey'):
        super().__init__(master)

        self.placeholder = placeholder
        self.placeholder_color = color
        self.default_fg_color = self['fg']

        self.bind("<FocusIn>", self.foc_in)
        self.bind("<FocusOut>", self.foc_out)

        self.put_placeholder()

    def put_placeholder(self):
        self.insert(0, self.placeholder)
        self['fg'] = self.placeholder_color

    def foc_in(self, *args):
        if self['fg'] == self.placeholder_color:
            self.delete('0', 'end')
            self['fg'] = self.default_fg_color

    def foc_out(self, *args):
        if not self.get():
            self.put_placeholder()



def openNewWindow(root):
    newWindow = Toplevel(root)
    # sets the title of the
    # Toplevel widget
    newWindow.title("Help Window")
    # sets the geometry of toplevel
    newWindow.geometry("850x400")
    # A Label widget to show in toplevel
    text1="In CLI you can type in below commands to perform different functions:"
    text2="-Create a directory: mkdir path/directoryname; eg: mkdir /user/john/"
    text3="-List all the files in director: ls path"
    text4="-Remove a directory: rm directory_path"
    text5="-Display all the contents of a file: cat path; eg /user/john/hello.txt"
    text6="-Uploading a file to the file system to a particular partition: put filename path partition#; put('data.csv','/user/john', 5)"
    text7="-Get the loaction of a file: getPartitionLocations path; eg: getPartitionLocations('/user/john/data.csv')"
    text8="-Read the contents of a prtition: readPartition path partition#; eg: readPartition('/user/shreya', 3)"
    text11="Search Methods"
    text9="-In order to find all the transactions occured by a cab company either Pink or Yellow cabs enter the cab company and from and to dates"
    text10="-To Search the city of customer enter customer ID in the text box"

    l1=Label(newWindow,text =text1)
    l1.place(x=5, y=5)
    l2=Label(newWindow,text =text2)
    l2.place(x=5, y=35)
    l3=Label(newWindow,text =text3)
    l3.place(x=5, y=65)
    l4=Label(newWindow,text =text4)
    l4.place(x=5, y=95)
    l5=Label(newWindow,text =text5)
    l5.place(x=5, y=125)
    l6=Label(newWindow,text =text6)
    l6.place(x=5, y=155)
    l7=Label(newWindow,text =text7)
    l7.place(x=5, y=185)
    l8=Label(newWindow,text =text8)
    l8.place(x=5, y=215)
    l11=Label(newWindow,text =text11)
    l11.place(x=5, y=260)
    l9=Label(newWindow,text =text9)
    l9.place(x=5, y=290)
    l10=Label(newWindow,text =text10)
    l10.place(x=5, y=310)

def view_tarnsactionID(cabcompany,date_from,date_to, root):
    result = search_tarnsactionID(cabcompany,date_from,date_to)
    new = Toplevel(root)
    new.geometry("750x250")
    new.title("Search1")
    table=pt.Table(new,dataframe=result,showtoolbar=True,showstatusbar=True)
    table.show()
#     Label(new, text=result).pack(pady=30)

def view_City(customerID, root):
    result = search_cityofCustomer(customerID)
    new = Toplevel(root)
    new.geometry("750x250")
    new.title("Search2")
#     table=pt.Table(new,dataframe=result,showtoolbar=True,showstatusbar=True)
#     table.show()
    Label(new, text=result).pack(pady=30)

    
def iterate_directory_structure(directory_json, dir_name, parent_dir, parent_iid):
    files = directory_json.keys()
    if '%ftype%' in files and directory_json['%ftype%']=='DIRECTORY':
        if parent_iid!=None:
            subdir_iid = treeview.insert(parent=parent_iid, index='end', text=dir_name)
        else:
            parent_iid = treeview.insert(parent='',index='0',text='root',open=True)
            subdir_iid = parent_iid
        for f in files:
            if f=='%ftype%':
                continue
            else:
                iterate_directory_structure(directory_json[f], f, dir_name, subdir_iid)
    else:
        file_name = dir_name
        if '%dot%' in dir_name:
            a = dir_name.rfind("%dot%")
            file_name = file_name[0:a]+"."+file_name[a+5:]
#         print(parent_iid, file_name)
        treeview.insert(parent=parent_iid, index='end', text=file_name)

    
    
def refreshDirStructure(treeview):
    r = requests.get("https://dsci551-337de-default-rtdb.firebaseio.com/.json")
    directory_json2 = r.json()
    for i in treeview.tvw_results.get_children():
        treeview.tvw_results.delete(i)
    iterate_directory_structure(directory_json2['root'], 'root', '', None)
    
r = requests.get("https://dsci551-337de-default-rtdb.firebaseio.com/.json")
directory_json = r.json()

root = tk.Tk()
root.title('DSCI 551 Project - EDFS')
root.config(width=800, height=600)

left_frame = Frame(root, width = 200, height=600, relief=tk.SUNKEN)
left_frame.pack(side=tk.LEFT, fill = tk.BOTH)
mid_frame = Frame(root, width = 450, height=600, relief=tk.SUNKEN)
mid_frame.pack(side=tk.RIGHT, fill = tk.BOTH)
top_frame = Frame(mid_frame, width = 450, height=150, relief=tk.SUNKEN)
top_frame.pack(side=tk.TOP, fill = tk.BOTH)
bottom_frame = Frame(mid_frame, width = 450, height=450, relief=tk.SUNKEN)
bottom_frame.pack(side=tk.BOTTOM, fill = tk.BOTH)

treeview = MyTreeView(left_frame)
refreshButton = Button(left_frame, text='Refresh', command=lambda: refreshDirStructure(treeview))

refreshButton.pack()
treeview.pack(fill='both', expand=True)

shell = Shell(top_frame, width=80, height=20, font=('Arial', 12), bg='black', fg='white')
shell.pack(fill=tk.BOTH, expand=1)

btn = Button(bottom_frame,text ="Help",command = lambda:openNewWindow(root))
btn.pack(pady = 10)
btn.place(x=5, y=10)

### Search UI

searchMainText=StringVar()
searchMainText.set("Search: ")
searchMainLabel=Label(bottom_frame, text="Search: ", height=1, font='Helvetica 18 bold')
searchMainLabel.place(x=5, y=40)
root.update()

search1Text=StringVar()
search1Text.set("1. Search for the transactions occured within a range of dates for a particular cab company.")
search1Label=Label(bottom_frame, text="1. Search for the transactions occured within a range of dates for a particular cab company.", height=1)
search1Label.place(x=5, y=70)
root.update()

selectMonth1Text=StringVar()
selectMonth1Text.set("Select cab Company")
selectMonth1Label=Label(bottom_frame, text="Select cab Company", height=1)
selectMonth1Label.place(x=20, y=100)

month1 = StringVar(bottom_frame)
month1.set("--Select--")
month1Menu = OptionMenu(bottom_frame, month1, "Pink Cab","Yellow Cab")
month1Menu.place(x=160, y=100)

selectdateText=StringVar()
selectdateText.set("Select from Date")
selectdateLabel=Label(bottom_frame, text="Select from Date", height=1)
selectdateLabel.place(x=20, y=130)


val2Entry =EntryWithPlaceholder(bottom_frame,"YYYY-MM-DD")
# val2Entry.insert(0,"YYYY-MM-DD")
val2Entry.place(x=160, y=130)

selecttodateText=StringVar()
selecttodateText.set("Select to Date")
selecttodateLabel=Label(bottom_frame, text="Select to Date", height=1)
selecttodateLabel.place(x=20, y=160)


val3Entry =EntryWithPlaceholder(bottom_frame,"YYYY-MM-DD")
# val3Entry.insert(0,"YYYY-MM-DD")
val3Entry.place(x=160, y=160)

search1Button = Button(bottom_frame, text='Submit', command=lambda: view_tarnsactionID(month1.get(),val2Entry.get(),val3Entry.get(),root))
search1Button.place(x=20, y=190)

search2Text=StringVar()
search2Text.set("2. Search the city of a customer")
search2Label=Label(bottom_frame, text="2. Search the city of a customer", height=1)
search2Label.place(x=5, y=230)

selectMonth2Text=StringVar()
selectMonth2Text.set("Enter Customer ID")
selectMonth2Label=Label(bottom_frame, text="Enter Customer ID", height=1)
selectMonth2Label.place(x=20, y=260)

month2Menu =EntryWithPlaceholder(bottom_frame,"0000")
# month2Menu.insert(0,"0000")
month2Menu.place(x=160, y=260)

search2Button = Button(bottom_frame, text='Submit', command=lambda: view_City(int(month2Menu.get()),root))
search2Button.place(x=20, y=290)

### End of search UI


iterate_directory_structure(directory_json['root'], 'root', '', None)

root.mainloop()

# enter on empty allows to enter cmd. should not allow

data.csv


KeyboardInterrupt: 